In [0]:
!pip install kaggle

In [0]:
!export KAGGLE_USERNAME=parthpartani
!export KAGGLE_KEY=f131feed2dd3ba88ad0100c3e5cdadbd

In [0]:
from google.colab import files

#retrieve uploaded file
uploaded = files.upload()

#print results
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [0]:
!kaggle competitions download -c career-con-2019

 82% 14.0M/17.1M [00:00<00:00, 27.3MB/s]
100% 17.1M/17.1M [00:00<00:00, 43.3MB/s]
  0% 0.00/64.1k [00:00<?, ?B/s]
100% 64.1k/64.1k [00:00<00:00, 19.9MB/s]
  0% 0.00/51.1k [00:00<?, ?B/s]
100% 51.1k/51.1k [00:00<00:00, 44.3MB/s]
 52% 9.00M/17.2M [00:00<00:00, 25.6MB/s]
100% 17.2M/17.2M [00:00<00:00, 43.3MB/s]


In [0]:
!unzip X_train.csv.zip
!unzip X_test.csv.zip

Archive:  X_train.csv.zip
  inflating: X_train.csv             
Archive:  X_test.csv.zip
  inflating: X_test.csv              


In [0]:
import numpy as np
import pandas as pd
import math

In [0]:
x_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")

In [0]:
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)
    return X,Y,Z

In [0]:
def feature_extraction(raw_frame):
    frame = pd.DataFrame()
    raw_frame['orientation'] = raw_frame['orientation_X'] + raw_frame['orientation_Y'] + raw_frame['orientation_Z']+ raw_frame['orientation_W']
    
    Ex, Ey, Ez = [], [], []
    x, y, z, w = raw_frame['orientation_X'].tolist(), raw_frame['orientation_Y'].tolist(), raw_frame['orientation_Z'].tolist(), raw_frame['orientation_W'].tolist()
    for i in range(len(x)):
        ex, ey, ez = quaternion_to_euler(x[i],y[i],z[i],w[i])
        Ex.append(ex)
        Ey.append(ey)
        Ez.append(ez)
    # raw_frame['eulerX'] = Ex
    # raw_frame['eulerY'] = Ey
    # raw_frame['eulerZ'] = Ez
    
    raw_frame['orientation_1'] = (raw_frame['orientation_X']**2 + raw_frame['orientation_Y']**2 + raw_frame['orientation_Z']**2 + raw_frame['orientation_W']**2)**0.5
    raw_frame['angular_velocity'] = raw_frame['angular_velocity_X'] + raw_frame['angular_velocity_Y'] + raw_frame['angular_velocity_Z']
    raw_frame['linear_acceleration'] = raw_frame['linear_acceleration_X']**2 + raw_frame['linear_acceleration_Y']**2 + raw_frame['linear_acceleration_Y']**2
    raw_frame['velocity_to_acceleration'] = raw_frame['angular_velocity'] / raw_frame['linear_acceleration']
    raw_frame['velocity_linear_acceleration'] = raw_frame['linear_acceleration'] * raw_frame['angular_velocity']

    for col in raw_frame.columns[3:]:
        frame[col + '_mean'] = raw_frame.groupby(['series_id'])[col].mean()
        frame[col + '_std'] = raw_frame.groupby(['series_id'])[col].std()
        frame[col + '_max'] = raw_frame.groupby(['series_id'])[col].max()
        frame[col + '_min'] = raw_frame.groupby(['series_id'])[col].min()
        frame[col + '_max_to_min'] = frame[col + '_max'] / frame[col + '_min']
        frame[col + '_mean_abs_change'] = raw_frame.groupby('series_id')[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        frame[col + '_abs_max'] = raw_frame.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
    return frame

In [0]:
trainm = feature_extraction(x_train)

In [0]:
trainm.shape

(3810, 112)

In [0]:
testm = feature_extraction(X_test)

In [0]:
label=pd.read_csv("y_train.csv")
label=label["surface"]

In [0]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(label)
label=label_encoder.transform(label)

In [0]:
import matplotlib.pyplot as plt 
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, f1_score, precision_recall_curve
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

In [0]:
Y_train = label
X_train = trainm
test = testm
X_train.shape, Y_train.shape
scaler = StandardScaler()
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(X_train)
test_img = scaler.transform(test)

In [0]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [0]:
clf1 = DecisionTreeClassifier(max_depth=10)
clf2 = KNeighborsClassifier(n_neighbors=10)
clf3 = SVC(gamma='scale', kernel='rbf', probability=True)
clf4 = xgb.XGBClassifier(
 learning_rate = 0.03,
 n_estimators= 100,
 max_depth= 150,
 min_child_weight= 2,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1)
clf5 = AdaBoostClassifier(n_estimators=600,learning_rate=1)
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3), ('xgb', clf4), ('ada', clf5)], voting='soft', weights=[2, 2, 2, 1, 1])

clf1 = clf1.fit(X_train, Y_train)
clf2 = clf2.fit(X_train, Y_train)
clf3 = clf3.fit(X_train, Y_train)
clf4 = clf4.fit(X_train, Y_train)
clf5 = clf5.fit(X_train, Y_train)
eclf = eclf.fit(X_train, Y_train)
weighted_preds = eclf.predict(test)

In [0]:
weighted_preds

array([4, 0, 7, ..., 1, 1, 8])

In [0]:
base = xgb.XGBClassifier(learning_rate = 0.03,n_estimators= 100,max_depth= 150,min_child_weight= 2,gamma=0.9,
                         subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread= -1,scale_pos_weight=1)

ada = AdaBoostClassifier(n_estimators=600,learning_rate=1)

model_ada = ada.fit(X_train, Y_train)

pred_ada = model_ada.predict(test)

In [0]:
pred_ada

array([4, 5, 1, ..., 1, 1, 8])

In [0]:
gbm = xgb.XGBClassifier(
 learning_rate = 0.03,
 n_estimators= 100,
 max_depth= 150,
 min_child_weight= 2,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(X_train, Y_train)
pred_xgb = gbm.predict(test)

In [0]:
pred_xgb

array([4, 0, 7, ..., 1, 1, 8])

In [0]:
import lightgbm
para={'boosting_type': 'gbdt',
 'colsample_bytree': 0.85,
 'learning_rate': 0.1,
 'max_bin': 512,
 'max_depth': -1,
 'metric': 'multi_error',
 'min_child_samples': 8,
 'min_child_weight': 1,
 'min_split_gain': 0.5,
 'nthread': 3,
 'num_class': 9,
 'num_leaves': 31,
 'objective': 'multiclass',
 'reg_alpha': 0.8,
 'reg_lambda': 1.2,
 'scale_pos_weight': 1,
 'subsample': 0.7,
 'subsample_for_bin': 200,
 'subsample_freq': 1}
from sklearn.model_selection import GridSearchCV,StratifiedKFold, train_test_split
kfold = StratifiedKFold(n_splits=5)
train_x,val_x,train_y,val_y = train_test_split(X_train, Y_train, test_size = 0.10, random_state=14)
gbm = xgb.XGBClassifier(learning_rate = 0.03,n_estimators= 100,max_depth= 150,min_child_weight= 2,gamma=0.9,subsample=0.8,
                        colsample_bytree=0.8,objective= 'binary:logistic',nthread= -1,scale_pos_weight=1)
for i in range(73):
    index = []
    for j in range(len(y_train)):
        if y_train.iloc[j]['group_id']==i:
            index.append(j)
    x_data = X_train.iloc[index]
    y_data = Y_train.iloc[index]
    gbm.fit(x_data, y_data)
#     model = lightgbm.train(para,train_data,valid_sets=test_data,num_boost_round=5000,early_stopping_rounds=70)
    #pred_lgbm = model.predict(test)
    #pred_lgbm = pd.DataFrame(pred_lgbm).idxmax(axis=1)

[1]	valid_0's multi_error: 0.60105
Training until validation scores don't improve for 70 rounds.
[2]	valid_0's multi_error: 0.404199
[3]	valid_0's multi_error: 0.307087
[4]	valid_0's multi_error: 0.251969
[5]	valid_0's multi_error: 0.212598
[6]	valid_0's multi_error: 0.212598
[7]	valid_0's multi_error: 0.19685
[8]	valid_0's multi_error: 0.19685
[9]	valid_0's multi_error: 0.186352
[10]	valid_0's multi_error: 0.175853
[11]	valid_0's multi_error: 0.175853
[12]	valid_0's multi_error: 0.173228
[13]	valid_0's multi_error: 0.167979
[14]	valid_0's multi_error: 0.173228
[15]	valid_0's multi_error: 0.165354
[16]	valid_0's multi_error: 0.15748
[17]	valid_0's multi_error: 0.154856
[18]	valid_0's multi_error: 0.15748
[19]	valid_0's multi_error: 0.146982
[20]	valid_0's multi_error: 0.144357
[21]	valid_0's multi_error: 0.141732
[22]	valid_0's multi_error: 0.128609
[23]	valid_0's multi_error: 0.141732
[24]	valid_0's multi_error: 0.141732
[25]	valid_0's multi_error: 0.139108
[26]	valid_0's multi_error:

In [0]:
pred_lgbm.shape

(3816,)

In [0]:
preds = (pred_ada + pred_xgb + pred_lgbm)/3
preds.shape

(3816,)

In [0]:
p = []
for i in range(len(preds)):
    p.append(int(math.floor(preds[i])))

In [0]:
class_prediction=pd.DataFrame(preds).idxmax(axis=1) 

In [0]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(max_depth = 10, random_state = 1)
abc = AdaBoostClassifier(base_estimator=dtree, n_estimators=600, learning_rate=1)
model_ada1 = abc.fit(X_train, Y_train)
pred_newada = model_ada1.predict(test)

In [0]:
dtree = DecisionTreeClassifier(max_depth = 20, random_state = 1)
abc = AdaBoostClassifier(base_estimator=dtree, n_estimators=600, learning_rate=1)
model1 = AdaBoostClassifier(base_estimator=abc, n_estimators=600, learning_rate=2)
model2 = AdaBoostClassifier(base_estimator=model1, n_estimators=1000, learning_rate=5)
model_ada1 = model2.fit(X_train, Y_train)
pred_2ada = model_ada1.predict(test)

In [0]:
pred_2ada

array([4, 1, 7, ..., 1, 1, 8])

In [0]:
submission1 = pd.DataFrame({
        "series_id": testm.index,
        "surface": weighted_preds,
        
    })
submission1.surface.value_counts()

1    988
5    750
8    593
6    457
7    376
4    319
2    185
0    146
3      2
Name: surface, dtype: int64

In [0]:
submission1 = pd.DataFrame({
        "series_id": testm.index,
        "surface": pred_newada,
        
    })
submission1.surface.value_counts()

1    1021
5     803
8     729
7     390
6     373
2     211
4     193
0      96
Name: surface, dtype: int64

In [0]:
submission1.surface=label_encoder.inverse_transform(submission1.surface)


In [0]:
submission1.surface.value_counts()

concrete                  988
soft_pvc                  750
wood                      593
soft_tiles                457
tiled                     376
hard_tiles_large_space    319
fine_concrete             185
carpet                    146
hard_tiles                  2
Name: surface, dtype: int64

In [0]:
submission1.to_csv('submission.csv', index=False)

In [0]:
!kaggle competitions submit -c career-con-2019 -f submission.csv -m "Submission_voting"

100% 53.6k/53.6k [00:02<00:00, 24.1kB/s]
Successfully submitted to CareerCon 2019 - Help Navigate Robots 